In [1]:
import os
import json
import base64
import re
import shutil
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [3]:
class TextCleaner:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
    
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text).strip()  # Supprimer les espaces inutiles
        text = re.sub(r'\d+', '', text)  # Supprimer les nombres
        text = text.lower()

        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]  # Lemmatisation et suppression des stopwords
        
        return " ".join(tokens)

In [4]:
 """Classe responsable de l'extraction et de l'enregistrement des images
    contenues dans les posts du forum sous forme de Base64."""
import os
import shutil
import base64

class ImageProcessor:

    def __init__(self, output_dir):
        """
        Initialise l'instance du processeur d'images.

        :param output_dir: Répertoire où les images extraites seront enregistrées.
        """
        self.output_dir = output_dir

        # Supprime le dossier de sortie s'il existe déjà, puis le recrée
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        os.makedirs(output_dir, exist_ok=True)

    def save_images(self, base64_images, prefix):
        """
        Enregistre les images en Base64 sous forme de fichiers PNG.

        :param base64_images: Liste des images encodées en Base64.
        :param prefix: Préfixe pour le nom des fichiers enregistrés.
        :return: Liste des noms de fichiers des images enregistrées.
        """
        filenames = []

        for idx, base64_img in enumerate(base64_images):
            try:
                # Décodage de l'image Base64
                img_data = base64.b64decode(base64_img)
                img_filename = f"{prefix}_{idx}.png"
                img_path = os.path.join(self.output_dir, img_filename)

                # Écriture du fichier image
                with open(img_path, "wb") as img_file:
                    img_file.write(img_data)

                filenames.append(img_filename)
            except Exception as e:
                print(f"⚠️ Erreur lors du décodage de l'image {idx}: {e}")

        return filenames


In [5]:
"""Classe représentant un post du forum, incluant le titre, l'URL, 
    la description et les solutions associées."""
class Post:
    def __init__(self, title, url, description, solutions):
        self.title = title
        self.url = url
        self.description = description
        self.solutions = solutions

In [6]:
import json
import re

class ForumPostProcessor:
    """Classe responsable du prétraitement des posts de forum, y compris le nettoyage du texte et la gestion des images."""

    def __init__(self, input_file, output_file, output_dir):
        """
        Initialise le processeur de posts du forum.

        :param input_file: Chemin du fichier JSON d'entrée contenant les posts.
        :param output_file: Chemin du fichier JSON de sortie pour stocker les posts traités.
        :param output_dir: Répertoire où seront enregistrées les images extraites.
        """
        self.input_file = input_file
        self.output_file = output_file
        self.cleaner = TextCleaner()
        self.image_processor = ImageProcessor(output_dir)
        
    def process_text_with_images(self, text, images):
        """
        Remplace les références aux images sous forme de texte par leurs chemins de fichiers réels.

        :param text: Texte contenant des références aux images (ex: "[Image_0]").
        :param images: Liste des chemins des images correspondantes.
        :return: Texte avec les références d'images remplacées par leurs chemins réels.
        """
        image_pattern = re.compile(r"\[Image_(\d+)\]")

        def replace_match(match):
            index = int(match.group(1))
            return f"[Image: {images[index]}]" if index < len(images) else match.group(0)

        return image_pattern.sub(replace_match, text)
        
    def process_post(self, post):
        """
        Traite un post du forum en nettoyant son texte et en gérant ses images.

        :param post: Dictionnaire contenant les données du post (titre, description, solutions...).
        :return: Un objet Post contenant le post traité.
        """
        description_text = post["description"].get("text", "") if isinstance(post["description"], dict) else post["description"]
        description_images = post["description"].get("images", []) if isinstance(post["description"], dict) else []
        
        description_text = self.cleaner.clean_text(description_text)
        description_text = self.process_text_with_images(description_text, description_images)
        
        solutions = []
        for sol_idx, solution in enumerate(post.get("solutions", [])):
            solution_text = self.cleaner.clean_text(solution["text"])
            solution_images = solution.get("images", [])
            extracted_images = self.image_processor.save_images(solution_images, f"img_{sol_idx}")
            solution_text = self.process_text_with_images(solution_text, extracted_images)
            solutions.append({"text": solution_text, "images": extracted_images})
        
        return Post(post["title"], post["url"], {"text": description_text, "images": description_images}, solutions)

    def process(self):
        """
        Lit le fichier JSON d'entrée, traite chaque post, puis enregistre les résultats dans un fichier JSON de sortie.
        """
        with open(self.input_file, 'r', encoding='utf-8') as f:
            forum_posts = json.load(f)
        
        processed_posts = [self.process_post(post).__dict__ for post in forum_posts]
        
        with open(self.output_file, "w", encoding="utf-8") as f:
            json.dump(processed_posts, f, ensure_ascii=False, indent=4)
        
        print(f"✅ Prétraitement terminé ! Données sauvegardées dans {self.output_file}")

# Exécution du traitement
input_json_file = "/kaggle/input/outsystemforums/processed_forum_data.json"
output_json_file = "/kaggle/working/processed_forum_data_cleaned.json"
output_dir = "/kaggle/working/images"

processor = ForumPostProcessor(input_json_file, output_json_file, output_dir)
processor.process()


✅ Prétraitement terminé ! Données sauvegardées dans /kaggle/working/processed_forum_data_cleaned.json
